## License
This file is part of the project megFingerprinting. All of megFingerprinting code is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version. megFingerprinting is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with megFingerprinting. If not, see https://www.gnu.org/licenses/.

In [8]:
import difflib
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import seaborn as sns
import scipy as sp
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import pearsonr
sns.set(font_scale=2)
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("husl", 8))
import math

In [9]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), sp.stats.sem(a)
    h = np.percentile(a, (1-((1-confidence)/2))*100)
    l = np.percentile(a, ((1-confidence)/2)*100)
    return m, l, h



## Bootstrap values of differentiation accuracy Controls

In [15]:
from random import sample
# Parameters
n_subs = 54 # Change here to get number of participants! 
n_feats = int(68*451)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:451].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraCTL/') if isfile(join('NEWspectraCTL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraCTL/' + iFile
    #print(sub)
    #print(sub[33])
    if sub[28] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        
        
n_subs=50
niter=1000
bootstrap = np.zeros([niter, 2])
self_id =np.zeros([n_subs, niter])

for j in range(0,niter):
    index=sample(range(54), n_subs)
    Tempsub_target=sub_target[index,:]
    Tempsub_database=sub_database[index,:]
    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(Tempsub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(Tempsub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (Tempsub_database.shape[1] - 1) # target, database
    bootstrap[j,0] = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    bootstrap[j,1] = accuracy_score(range(n_subs), predictions.argmax(axis = 0))
    self_id[0:n_subs,j] = np.diagonal(sp.stats.zscore(predictions, axis = 1))

print(mean_confidence_interval(bootstrap.flatten()))

print(mean_confidence_interval(self_id.flatten()))

(0.9013000000000002, 0.88, 0.94)
(2.9877339930181672, 0.9729036783198106, 4.4771913149606375)


In [16]:
from random import sample
# Parameters
n_subs = 79 # Change here to get number of participants! 
n_feats = int(68*451)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:451].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraPKD/') if isfile(join('NEWspectraPKD/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraPKD/' + iFile
    #print(sub)
    #print(sub[33])
    if sub[28] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        
        
n_subs=70
niter=1000
bootstrap = np.zeros([niter, 2])
self_id =np.zeros([n_subs, niter])

for j in range(0,niter):
    index=sample(range(79), n_subs)
    Tempsub_target=sub_target[index,:]
    Tempsub_database=sub_database[index,:]
    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(Tempsub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(Tempsub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (Tempsub_database.shape[1] - 1) # target, database
    bootstrap[j,0] = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    bootstrap[j,1] = accuracy_score(range(n_subs), predictions.argmax(axis = 0))
    self_id[0:n_subs,j] = np.diagonal(sp.stats.zscore(predictions, axis = 1))

print(mean_confidence_interval(bootstrap.flatten()))

print(mean_confidence_interval(self_id.flatten()))

(0.7806500000000002, 0.7428571428571429, 0.8285714285714286)
(2.621978089768564, 0.31333489490631034, 4.4058416861660925)


In [10]:
# Parameters
from random import sample
n_subs = 133 # Change here to get number of participants! 
n_feats = int(68*451)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    #print(filename[19:23])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:451].flatten()
    sub_feat[0, -1] = int(filename[19:23])    
    return sub_feat


# Get n subjects: both training and testing datasets
onlyfiles = [f for f in listdir('NEWspectraFUL/') if isfile(join('NEWspectraFUL/', f))]
sub_target = np.zeros([n_subs, (n_feats)+1])
sub_database = np.zeros([n_subs, (n_feats)+1])
iv = 0
it = 0
for iFile in sorted(onlyfiles)[0:(n_subs*2)]: 
    sub = 'NEWspectraFUL/' + iFile
    #print(sub)
    #print(sub[28])
    if sub[28] == 'v':
        sub_target[iv, :] = prune_subject_csv(sub)
        iv += 1
    else:
        sub_database[it, :] = prune_subject_csv(sub)
        it += 1
        
        
n_subs=133
n_subs_2=51
niter=1000
acc_over=np.zeros([niter,2])

for j in range(0,niter):
    self_acc=np.zeros([133,2])
    for i in range(54,133):
        np_arr1=np.array(i)
        np_arr2= sample(range(0,54), 50)
        index=np.append(np_arr2, np_arr1)

        sub_target_1=sub_target[index, :]
        sub_database_1=sub_database[index, :]

        # Correlations can be computed as the dot product between two z-scored vectors
        z_target = sp.stats.zscore(sub_target_1[:, :-1], axis = 1)
        z_database = sp.stats.zscore(sub_database_1[:,:-1], axis = 1)
        predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
        target_from_database = accuracy_score(range(n_subs_2), predictions.argmax(axis = 1))
        database_from_target = accuracy_score(range(n_subs_2), predictions.argmax(axis = 0))
        
        self_id = np.diagonal(sp.stats.zscore(predictions, axis = 1))
        cor_predictions=np.diagonal(predictions)     
        self_acc[i,0]= (predictions.argmax(axis = 1)[-1]==50)
        self_acc[i,1]= (predictions.argmax(axis = 0)[-1]==50)
        #print(self_acc)
    acc_over[j,0]=  (sum(self_acc)/79)[0]
    acc_over[j,1]=  (sum(self_acc)/79)[1]
            
print(mean_confidence_interval(acc_over.flatten()))

(0.8146772151898738, 0.810126582278481, 0.8354430379746836)
